In [2]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.8/220.8 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [29]:
import pandas as pd
import random

from openai import OpenAI

def generate_skus_and_descriptions(n, types):
    skus = []
    sku_types = []
    descriptions = []
    num_skus_per_type = n // len(types)

    client = OpenAI(api_key="")

    for product_type in types:
        # Generate product names
        prompt = f"Generate {num_skus_per_type} product names for the category '{product_type}':"
        response = client.chat.completions.create(
            model="gpt-3.5-turbo-1106",
            messages=[{'role': 'user', 'content': prompt}]
        )

        response_text = response.choices[0].message.content
        generated_names = response_text.split('\n')
        cleaned_names = [name.strip().split(' ', 1)[1] if ' ' in name else name.strip() for name in generated_names if name.strip()]
        valid_names = cleaned_names[:num_skus_per_type]
        skus.extend(valid_names)
        sku_types.extend([product_type] * len(valid_names))

        print(product_type)
        print(response)

        # Generate descriptions for each product
        for sku in valid_names:
            description_prompt = f"Write a short description for a {product_type} product named '{sku}':"
            description_response = client.chat.completions.create(
                model="gpt-3.5-turbo-1106",
                messages=[{'role': 'user', 'content': description_prompt}]
            )

            description_text = description_response.choices[0].message.content
            descriptions.append(description_text.strip())

    return skus[:n], sku_types[:n], descriptions[:n]



# Other attributes
types = ["Dairy", "Bread", "Fruit", "Produce", "Condiment"]
vegan_options = ["Vegan", "Non-Vegan"]
gluten_options = ["Gluten-Free", "Non-Gluten-Free"]

# Generate SKU names and descriptions
num_products = 20
sku_names, sku_types, sku_descriptions = generate_skus_and_descriptions(num_products, types)

# Generate data for other columns
data = {
    "SKU": sku_names,
    "Type": sku_types,
    "Description": sku_descriptions,
    "Cost": [f"${random.uniform(1, 5):.2f}" for _ in range(num_products)],
    "Quantity (g)": [random.randint(100, 1000) for _ in range(num_products)],
    "Vegan/Non-Vegan": [random.choice(vegan_options) for _ in range(num_products)],
    "Gluten-Free/Non-Gluten-Free": [random.choice(gluten_options) for _ in range(num_products)]
}


# Create DataFrame
df = pd.DataFrame(data)

# Function to remove the initial number from SKU names
def remove_initial_number(sku):
    return ' '.join(sku.split(' ')[1:])

# Apply the function to the 'SKU' column of the DataFrame
df['SKU'] = df['SKU'].apply(remove_initial_number)


print(df)


Dairy
ChatCompletion(id='chatcmpl-8ObiOT1o0OxHug8lECCihxXIqHqkN', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='1. Creamy Moo Milk\n2. Cheesy Delight Cheese\n3. Butterlicious Spread\n4. Yogurt Blissful Delight', role='assistant', function_call=None, tool_calls=None))], created=1700875540, model='gpt-3.5-turbo-1106', object='chat.completion', system_fingerprint='fp_eeff13170a', usage=CompletionUsage(completion_tokens=30, prompt_tokens=19, total_tokens=49))
Bread
ChatCompletion(id='chatcmpl-8ObibcqNF50UELgeKRxmVnwEc1Fb2', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='1. Sourdough Supreme\n2. Whole Wheat Wonder\n3. Artisanal Harvest\n4. Buttery Brioche', role='assistant', function_call=None, tool_calls=None))], created=1700875553, model='gpt-3.5-turbo-1106', object='chat.completion', system_fingerprint='fp_eeff13170a', usage=CompletionUsage(completion_tokens=27, prompt_tokens=19, total_tokens=46))
Fruit
ChatCo